In [1]:
import numpy as np
import numpy.linalg as lin
import scipy.stats as sts
import scipy.integrate as intgr
import scipy.optimize as opt
import scipy.special as ss
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import time
M = np.loadtxt("MacroSeries.txt", delimiter=",")

# M is structured as (c, k, w, r)

In [2]:
sigma = 0.1

mu_i = 1.1
beta_i = 0.8
rho_i = 0.15
alpha_i = 0.5
pmi = np.array([mu_i,beta_i,rho_i,alpha_i])

def GMMprob4(params,M,sigma):
    mu,beta,rho,alpha = params
    Z = np.log(M[:,2]/((1-alpha)*(M[:,1]**alpha)))
    
    mom_1 = Z[1:] - rho*Z[:-1] - (1 - rho)*mu
    mom_2 = (Z[1:] - rho*np.mean(Z[:-1]) - (1 - rho)*mu)*Z[:-1]
    mom_3 = beta*alpha*np.exp(Z[1:])*(M[1:,1]**(alpha-1))*(M[:-1,0]/M[1:,0]) - 1
    mom_4 = (beta*alpha*np.exp(Z[1:])*(M[1:,1]**(alpha-1))*(M[:-1,0]/M[1:,0]) - 1)*M[:-1,2]
    
    m1 = np.mean(mom_1)
    m2 = np.mean(mom_2)
    m3 = np.mean(mom_3)
    m4 = np.mean(mom_4)
    
    errors = -np.array([m1,m2,m3,m4])
    err_fin= np.dot(errors,errors)
    return err_fin

We seek to estimate the problem in #2, but with GMM.

In [3]:
bounds = ((1e-10,None),(1e-10,None),(-1,1),(0,1))

gmm_args = (M,sigma)
start = time.time()
results = opt.minimize(GMMprob4, pmi, args=(gmm_args),bounds=bounds)
print("Total Algorithm Duration: ",time.time()-start,"seconds")
print("Final Minimized Error   : ",-results.fun)
print("   ")
mumle, betamamle, rhomle, alphamle  = results.x
print("Mu (GMM)    :",mumle)
print("Beta (GMM)  :",betamamle)
print("Rho (GMM)   :",rhomle)
print("Alpha (GMM) :",alphamle)
print(results.hess_inv)

Total Algorithm Duration:  0.08804202079772949 seconds
Final Minimized Error   :  -4025.37106678
   
Mu (GMM)    : 1.10000000001
Beta (GMM)  : 0.770942816236
Rho (GMM)   : 0.150000000007
Alpha (GMM) : 0.481837331811
<4x4 LbfgsInvHessProduct with dtype=float64>
